In [1]:
import pandas as pd
import pickle
import numpy as np
import torch
from torch import nn
import os 
import fastai
import fastai
from fastai.text import *
PATH="/home/kirana/Documents/phd/final"
DATAPATH="/home/kirana/Documents/phd/data/aclImdb"

## Dataset

In [2]:
[df_train,df_valid,df_test,itos, train_tokens, valid_tokens, test_tokens, trn_lm, val_lm, test_lm]=pickle.load(open(f'{PATH}/inter/dfs_tokens_fastai.pkl','rb'))

In [3]:
type(df_train)

pandas.core.frame.DataFrame

In [4]:
df_train=pd.concat([df_train,df_valid],axis=0)

In [5]:
df_train.shape

(75000, 4)

In [6]:
df_valid=df_test.copy()

In [7]:
df_train=df_train.loc[df_train['label']<2]

In [8]:
df_train.shape

(25000, 4)

In [9]:
df_train.head()

,label,text,words,tokens
3218,0,I've noticed how all the other reviews of this...,"[ \n , xxbos, xxfld, 1, i, 've, noticed, how, ...","[41, 42, 43, 40, 13, 161, 2030, 110, 44, 3, 10..."
6854,0,I went on a visit to one of my relatives a whi...,"[ \n , xxbos, xxfld, 1, i, went, on, a, visit,...","[41, 42, 43, 40, 13, 436, 30, 7, 2074, 9, 39, ..."
2697,0,Darcy and her young daughter Pamela are headin...,"[ \n , xxbos, xxfld, 1, xxmaj, darcy, and, her...","[41, 42, 43, 40, 2, 9225, 6, 55, 207, 551, 2, ..."
5747,0,Battlestar Gallactica was so great because it ...,"[ \n , xxbos, xxfld, 1, xxmaj, battlestar, xxm...","[41, 42, 43, 40, 2, 7900, 2, 33242, 20, 52, 10..."
8731,0,"Any movie with ""National Lampoon"" in the title...","[ \n , xxbos, xxfld, 1, xxmaj, any, movie, wit...","[41, 42, 43, 40, 2, 120, 25, 23, 16, 2, 2155, ..."


In [10]:
type(df_train['tokens'])

pandas.core.series.Series

In [11]:
bs=52 # 52 - Jeremey, 20 - default

In [12]:
bptt= 70 #70 - Jeremey, 35 - default

In [13]:
df_train.head()

,label,text,words,tokens
3218,0,I've noticed how all the other reviews of this...,"[ \n , xxbos, xxfld, 1, i, 've, noticed, how, ...","[41, 42, 43, 40, 13, 161, 2030, 110, 44, 3, 10..."
6854,0,I went on a visit to one of my relatives a whi...,"[ \n , xxbos, xxfld, 1, i, went, on, a, visit,...","[41, 42, 43, 40, 13, 436, 30, 7, 2074, 9, 39, ..."
2697,0,Darcy and her young daughter Pamela are headin...,"[ \n , xxbos, xxfld, 1, xxmaj, darcy, and, her...","[41, 42, 43, 40, 2, 9225, 6, 55, 207, 551, 2, ..."
5747,0,Battlestar Gallactica was so great because it ...,"[ \n , xxbos, xxfld, 1, xxmaj, battlestar, xxm...","[41, 42, 43, 40, 2, 7900, 2, 33242, 20, 52, 10..."
8731,0,"Any movie with ""National Lampoon"" in the title...","[ \n , xxbos, xxfld, 1, xxmaj, any, movie, wit...","[41, 42, 43, 40, 2, 120, 25, 23, 16, 2, 2155, ..."


In [14]:
df_train['label'].value_counts()

1    12500
0    12500
Name: label, dtype: int64

In [15]:
df_valid['label'].value_counts()

1    12500
0    12500
Name: label, dtype: int64

In [16]:
itos[:5]

['_unk_', '_pad_', 'xxmaj', 'the', '.']

In [17]:
padlen=1400
padding_idx=1

In [18]:
df_train['n_tok']=df_train['tokens'].apply(len)

In [19]:
df_train['n_tok'].describe()

count    25000.000000
mean       304.945840
std        224.981807
min         16.000000
25%        166.000000
50%        228.000000
75%        371.000000
max       3354.000000
Name: n_tok, dtype: float64

In [20]:
df_train.sort_values(by='n_tok', inplace=True)

In [21]:
df_valid['n_tok']=df_valid['tokens'].apply(len)
df_valid.sort_values(by='n_tok', inplace=True)

In [22]:
df_test['n_tok']=df_test['tokens'].apply(len)
df_test.sort_values(by='n_tok', inplace=True)

In [23]:
for i in range(0,20):
    x=df_train['tokens'].values[i]
    y=df_train['label'].values[i]    
    out=np.ones(padlen)
    if len(x) < padlen:
        out[:len(x)]=x
    else:
        out=x[:padlen]
    print (out)
    print (y)
    print (len(out))
    if i>10:
        break

[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
1
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
1
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
1
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400


In [24]:
df_train.shape

(25000, 5)

In [25]:
df_train['label'].value_counts(), df_valid['label'].value_counts(),df_test['label'].value_counts()

(1    12500
 0    12500
 Name: label, dtype: int64, 1    12500
 0    12500
 Name: label, dtype: int64, 1    12500
 0    12500
 Name: label, dtype: int64)

In [26]:
df_train.shape, df_valid.shape

((25000, 5), (25000, 5))

In [27]:
padlen

1400

In [28]:
class ds_sentiment (Dataset):
    def __init__ (self,df,bs,padlen=64,xvar='tokens',yvar='label',len_var='n_tok',padding_idx=1):
        self.x,self.y,self.padlen,self.padding_idx,self.len_var,self.bs=\
            df[xvar],df[yvar],padlen,padding_idx,df[len_var],bs
        self.len_var=self.len_var.clip(0,padlen)
    
    def pad (self,x):
        out=np.ones(self.padlen)*self.padding_idx
        out=out.astype(int)
        if len(x)>=self.padlen:
            out[:]=x[:self.padlen]
        else:
            out[:len(x)]=x
        return out
        
    def __getitem__(self,idx):
        #import pdb
        #pdb.set_trace()
        return self.pad(self.x.iloc[idx]),self.y.iloc[idx],self.len_var.iloc[idx]
    
    def __len__(self):
        return len(self.x)        


In [29]:
#padlen=max(df_train['n_tok'])
padlen=1400

In [30]:
padlen

1400

In [31]:
dstrain=ds_sentiment(df_train,bs,padlen)
dsvalid=ds_sentiment(df_valid,bs,padlen)
dstest=ds_sentiment(df_test,bs,padlen)

In [32]:
dltrain=DataLoader(dstrain,bs,True)
dlvalid=DataLoader(dsvalid,bs,False)
dltest=DataLoader(dstest,bs,False)

In [33]:
for xb,yb,xlen in dltrain:
    break

In [34]:
xb, yb, xlen

(tensor([[41, 42, 43,  ...,  1,  1,  1],
         [41, 42, 43,  ...,  1,  1,  1],
         [41, 42, 43,  ...,  1,  1,  1],
         ...,
         [41, 42, 43,  ...,  1,  1,  1],
         [41, 42, 43,  ...,  1,  1,  1],
         [41, 42, 43,  ...,  1,  1,  1]]),
 tensor([1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
         0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0,
         0, 0, 0, 0]),
 tensor([ 147,  232,  176,  409,  230,  192,  250,  418,  178,  335,  242, 1095,
          114,  194,  159,  278,  285,  435,  428,  442,  201,  177,   81,  371,
          139,  199,  466,  176,  173,  182,  200,  355,   99,  235,  599,  225,
         1277,   94,   75,  175,  252,  174,  309,  473,  153,  140,  168,  453,
          166,  198,  106,  423]))

## Model Architecture

In [35]:
from torch import nn
import torch
from torch.autograd import Variable

In [36]:
bs,bptt

(52, 70)

In [37]:
n_inp=len(itos)
n_emb=400 #650
n_hidden=650#400
n_layers= 3 # 2
dropout=0.5 # 0.5
wd=1e-7
bidirectional=True
dropout_e=0.2 # 0.5
dropout_o=0.5 #0.5
n_out=1

In [38]:
xb.shape,xlen.shape

(torch.Size([52, 1400]), torch.Size([52]))

In [39]:
df_train['label'].value_counts()

1    12500
0    12500
Name: label, dtype: int64

In [40]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [41]:
itos[:5]

['_unk_', '_pad_', 'xxmaj', 'the', '.']

In [42]:
import torch

In [43]:
m = nn.AdaptiveAvgPool1d(5)
input = torch.randn(1, 64, 8)
input.size()

torch.Size([1, 64, 8])

In [44]:
m(input).size()

torch.Size([1, 64, 5])

In [45]:
class sentiment_classifier (nn.Module):
    def __init__(self,n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e=0.05,dropout=0.5,\
                 dropout_o=0.5,pretrain_mtx=None,n_out=1,padding_idx=1,n_filters=100,filter_sizes=[3,4,5]):
        super().__init__()
        self.n_inp,self.n_emb,self.n_hidden,self.n_layers,self.bidirectional,self.bs,self.device,self.pretrain_mtx,self.padding_idx=\
                            n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,pretrain_mtx,padding_idx
        self.n_out,self.n_filters,self.filter_sizes=n_out,n_filters,filter_sizes
        self.dropout_e,self.dropout,self.dropout_o=dropout_e,dropout,dropout_o
        
        self.create_architecture()
        if pretrain_mtx is not None:
            print (f'initializing glove with {pretrain_mtx.shape}')
            self.initialize_glove()
        self.init_hidden()
        self.criterion=nn.BCEWithLogitsLoss()
    
    def set_dropouts(self, dropout, dropout_o, dropout_e):
        self.dropout, self.dropout_o, self.dropout_e = dropout, dropout_o, dropout_e
    
    
    def freeze_embedding(self):
        self.encoder.weight.requires_grad=False
         
    def unfreeze_embedding(self):
        self.encoder.weight.requires_grad=True

    def initialize_glove(self):
        self.encoder.weight.data.copy_(torch.Tensor(self.pretrain_mtx))
    
    def init_hidden(self):
        # Initialize hidden
        self.hidden=(Variable(torch.zeros(self.n_layers,self.bs,self.n_hidden,requires_grad=False).to(self.device)),
                     Variable(torch.zeros(self.n_layers,self.bs,self.n_hidden,requires_grad=False).to(self.device)))
    

    def create_architecture(self):
        ###################################
        # Embedding layer - common to both
        ###################################
        self.dropout_enc=nn.Dropout(self.dropout_e)
        self.encoder=nn.Embedding(self.n_inp,self.n_emb,padding_idx=self.padding_idx)
        
        #######################################
        # For RNN #############################
        #######################################
        # Embedding Layer: Embedding layer just maps each word to an index. n_inp to n_emb mapping is all it does
            # input to this is of shape n_batch * n_seq
         # LSTM Layer
        self.lstm=nn.LSTM(self.n_emb,self.n_hidden,self.n_layers,batch_first=True,dropout=self.dropout,\
                          bidirectional=self.bidirectional)
          # embs are going to be of shape n_batch * n_seq * n_emb
        self.dropout_op=nn.Dropout(self.dropout_o)
        
        self.avg_pool1d=torch.nn.AdaptiveAvgPool1d(1)
        self.max_pool1d=torch.nn.AdaptiveMaxPool1d(1)
        
    
        #######################################
        # For CNN #############################
        #######################################    
        #embedding dimension is the "depth" of the filter and the number of tokens in the sentence is the width.
        self.conv_0=torch.nn.Conv1d (self.n_emb,self.n_filters,kernel_size=self.filter_sizes[0])
        self.conv_1=torch.nn.Conv1d (self.n_emb,self.n_filters,kernel_size=self.filter_sizes[1])
        self.conv_2=torch.nn.Conv1d(self.n_emb,self.n_filters,kernel_size=self.filter_sizes[2])
        
        self.fc=nn.Linear(len(self.filter_sizes)*self.n_filters+self.n_hidden*4,self.n_out)
        

        
    def forward (self,Xb,Yb,Xb_lengths):
        
        ####RNN PORTION
        embs=self.dropout_enc(self.encoder(Xb))
        if Xb.size(0) < self.bs:
            self.hidden=(self.hidden[0][:,:Xb.size(0),:].contiguous(),
            self.hidden[1][:,:Xb.size(0),:].contiguous())
        packed_embs = pack_padded_sequence(embs,Xb_lengths,batch_first=True, enforce_sorted=False)
        lstm_out,(hidden,cell)=self.lstm(packed_embs)
        lstm_out,lengths=pad_packed_sequence(lstm_out,batch_first=True)
        hidden = self.dropout_op(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        avg_pool=self.avg_pool1d(lstm_out.permute(0,2,1)).view(Xb.size(0),-1)
        max_pool=self.max_pool1d(lstm_out.permute(0,2,1)).view(Xb.size(0),-1)
        
        #CNN Portion
        new_embs=embs.permute(0,2,1)        
        conved_0=torch.relu(self.conv_0(new_embs))
        conved_1=torch.relu(self.conv_1(new_embs))
        conved_2=torch.relu(self.conv_2(new_embs)) 
        max_pool1d=torch.nn.MaxPool1d(conved_0.shape[2])
        pooled_0=max_pool1d(conved_0).squeeze(2)
        max_pool1d=torch.nn.MaxPool1d(conved_1.shape[2])
        pooled_1=max_pool1d(conved_1).squeeze(2)
        max_pool1d=torch.nn.MaxPool1d(conved_2.shape[2])
        pooled_2=max_pool1d(conved_2).squeeze(2)
        cat_cnn = self.dropout_op(torch.cat([pooled_0,pooled_1,pooled_2],dim=1))
        
        ## Concatenate
        big_out=torch.cat([cat_cnn,hidden,max_pool],dim=1)
        preds=self.fc(big_out)

        loss=self.criterion(preds,Yb.contiguous().float().view(-1,1))

        
        return preds.view(-1),loss
        
    

In [46]:
def accuracy_binomial(preds,actual, device="cpu", cutoff=0.5):
    preds=torch.sigmoid(preds)
    zeros=torch.zeros(len(preds)).to(device)
    ones = torch.ones(len(preds)).to(device)

    preds=torch.where(preds>cutoff,ones,zeros)
    correct=torch.round(preds).long()==actual
    return correct.float().sum()/len(correct)

In [47]:
def accuracy_binomial(preds, y, device="cpu", cutoff=0.5):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    y=y.float()
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [48]:
device="cpu"

## Load pretrained model weights

In [49]:
if 1==0:
    model=torch.load (f'{PATH}/inter/varybptt_model_awd_lstm')
    pretrained_lm_weights=model.encoder.weight.data.cpu().numpy()
    import pickle
    pickle.dump(pretrained_lm_weights,open(f'{PATH}/inter/varybpttpretrained_lm_weights','wb'))

In [50]:
pretrained_lm_weights=pickle.load(open(f'{PATH}/inter/varybpttpretrained_lm_weights','rb'))

### Check if model forward works

In [51]:
n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e,dropout_o,dropout,n_out

(59972, 400, 650, 3, True, 52, 'cpu', 0.2, 0.5, 0.5, 1)

In [52]:
model_sentiment=sentiment_classifier (n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e=dropout_e,dropout=dropout,\
                 dropout_o=dropout_o,pretrain_mtx=pretrained_lm_weights,n_out=1,padding_idx=1)


initializing glove with (59972, 400)


In [53]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {count_parameters(model_sentiment):,} trainable parameters')

The model has 50,243,201 trainable parameters


In [54]:
model_sentiment

sentiment_classifier(
  (dropout_enc): Dropout(p=0.2)
  (encoder): Embedding(59972, 400, padding_idx=1)
  (lstm): LSTM(400, 650, num_layers=3, batch_first=True, dropout=0.5, bidirectional=True)
  (dropout_op): Dropout(p=0.5)
  (avg_pool1d): AdaptiveAvgPool1d(output_size=1)
  (max_pool1d): AdaptiveMaxPool1d(output_size=1)
  (conv_0): Conv1d(400, 100, kernel_size=(3,), stride=(1,))
  (conv_1): Conv1d(400, 100, kernel_size=(4,), stride=(1,))
  (conv_2): Conv1d(400, 100, kernel_size=(5,), stride=(1,))
  (fc): Linear(in_features=2900, out_features=1, bias=True)
  (criterion): BCEWithLogitsLoss()
)

In [55]:
model_sentiment.forward(xb,yb,xlen)

(tensor([-0.1720,  0.2152,  0.1471,  0.0471,  0.2479, -0.1711, -0.1465, -0.0710,
          0.1347, -0.1875, -0.0061,  0.1740,  0.1584, -0.0970, -0.0822, -0.0072,
          0.1839,  0.0710,  0.0609, -0.1447,  0.1054,  0.1393,  0.0952,  0.1834,
          0.0654,  0.1726, -0.0235,  0.0480,  0.1059,  0.0055, -0.0384,  0.0519,
          0.0906,  0.0687,  0.1459, -0.0274, -0.0706,  0.0401,  0.1523,  0.1061,
          0.0200, -0.0222,  0.1844, -0.0557, -0.0227,  0.0875, -0.0396,  0.2745,
          0.2360,  0.2532,  0.2390,  0.0311], grad_fn=<ViewBackward>),
 tensor(0.7132, grad_fn=<BinaryCrossEntropyWithLogitsBackward>))

In [56]:
preds,loss=model_sentiment.forward(xb,yb,xlen)

In [57]:
preds.view(-1)

tensor([ 0.0656,  0.0136, -0.1372,  0.2664,  0.0462,  0.0115, -0.1068,  0.1533,
        -0.0305,  0.1814,  0.1157,  0.1596,  0.0667, -0.1568,  0.1733,  0.1093,
        -0.1035, -0.1171, -0.0759, -0.0396, -0.1533,  0.0909,  0.0184,  0.0966,
        -0.1293,  0.3231,  0.1233, -0.2267,  0.0190,  0.0853, -0.0648,  0.1876,
         0.2022, -0.0178,  0.2353,  0.1190,  0.1483,  0.1013,  0.1072,  0.2681,
         0.1409,  0.1748, -0.0869,  0.1414,  0.0591,  0.0903,  0.1532,  0.0349,
         0.0477,  0.1788, -0.0938,  0.2276], grad_fn=<ViewBackward>)

In [58]:
yb

tensor([1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
        0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0])

In [59]:
preds.size()

torch.Size([52])

In [60]:
yb.size()

torch.Size([52])

In [61]:
accuracy_binomial(preds.to(device),yb.to(device))

tensor(0.4231)

In [62]:
from sklearn.metrics import roc_auc_score

In [63]:
roc_auc_score(yb,preds)

0.4609250398724083

In [64]:
pretrained_lm_weights

array([[-0.145316,  0.061101, -0.033757,  0.046711, ..., -0.058453,  0.008306,  0.083778, -0.039654],
       [-0.022339, -0.025568,  0.051964, -0.041687, ..., -0.009573,  0.103125,  0.010123,  0.003203],
       [ 0.106638, -0.067644, -0.106292,  0.187177, ...,  0.035562,  0.054249,  0.158896,  0.07562 ],
       [ 0.126048,  0.231132, -0.045351,  0.129894, ..., -0.02568 ,  0.022782,  0.360755,  0.066252],
       ...,
       [ 0.023593,  0.005543,  0.223388, -0.111507, ..., -0.072671,  0.084183,  0.033837, -0.05104 ],
       [-0.118694,  0.003163,  0.080704, -0.06    , ...,  0.023115,  0.100214, -0.068435, -0.065693],
       [ 0.030997,  0.019074,  0.051958, -0.069346, ..., -0.086905,  0.150055,  0.035902, -0.072329],
       [ 0.002274, -0.002966,  0.049782, -0.051592, ..., -0.086745, -0.014879, -0.034296, -0.030868]], dtype=float32)

## Learner

In [65]:
# Weight Decay Schedule
tempstart=1e-6
for i in range (6):
    print (i, tempstart)
    tempstart=tempstart*9

0 1e-06
1 9e-06
2 8.1e-05
3 0.000729
4 0.006561000000000001
5 0.059049000000000004


In [66]:
tempstart=5e-3
for i in range (6):
    print (i, tempstart)
    tempstart=tempstart*0.5

0 0.005
1 0.0025
2 0.00125
3 0.000625
4 0.0003125
5 0.00015625


In [67]:
torch.randint(1,5,(1,))

tensor([1])

In [68]:
lrs=[1e-2,5e-3,1e-4,5e-4]

In [69]:
torch.randint(0,4,(1,))

tensor([2])

In [70]:
lrs[torch.randint(0,4,(1,))]

0.005

In [71]:
class Learner():
    def __init__(self,model,optimizer,metric_fn,device,bptt=12,print_every=5,clip_val=None,\
                 cycle_mult=0,lr_decay=0.7,wd_mult=6):
        self.model,self.optimizer,self.metric_fn,self.device,self.print_every,self.bptt,self.losses,self.clip_val=\
            model,optimizer,metric_fn,device,print_every,bptt,[],clip_val
        self.n_epochs=1
        self.cycle_mult,self.lr_decay=cycle_mult,lr_decay
        self.wd_mult=wd_mult
        for param_group in self.optimizer.param_groups:
            self.start_lr=param_group['lr']
            self.start_wd=param_group['weight_decay']
        self.wd=self.start_wd
        self.lr=self.start_lr
        self.n_epoch=0
        self.lrs=[1e-2,5e-3,1e-4,5e-4]
        
    def fit (self,Xb,Yb,Xlen,mode_train=True):
        if mode_train:
            self.model.train()
        else:
            self.model.eval()
            
        preds,loss=self.model(Xb,Yb,Xlen)
        
        with torch.no_grad():
            acc=self.metric_fn(preds,Yb.view(-1),self.device)
            acc=acc.item()
            del preds
        
        if mode_train:
            if 1==0:
                lr =self.lrs[torch.randint(0,4,(1,))]
                for param_group in self.optimizer.param_groups:
                    param_group['lr']=lr
            
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
        myloss=loss.item()
        del loss
        
        if self.clip_val is not None:
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.clip_val)
        
        return myloss, acc
    
    def lr_find (self,start_lr,end_lr,iterator,n_batch):
        losses,lrs=[],[]
        ratio=end_lr/start_lr
        num_steps=n_batch
        lr=start_lr
        for i in range(num_steps):            
            lr=lr*(end_lr/start_lr)**(1/num_steps)
            lrs.append(lr)
        self.lrs=lrs
        self.run_epoch(iterator,mode_train=True,lrs=lrs)
    
    def run_epoch(self,iterator,mode_train,lrs=None):
        epoch_loss,epoch_acc,i,k=0,0,0,0
        self.model.init_hidden()
        for Xb,Yb,Xlen in iterator:
            Xb=Xb.to(self.device)
            Yb=Yb.to(self.device)
            Xlen=Xlen.to(self.device)
            
            if lrs is not None:
                lr=lrs[k]
                for param_group in self.optimizer.param_groups:
                    param_group['lr']=lr 
            

            loss,acc=self.fit(Xb,Yb,Xlen,mode_train)
            
            if lrs is not None:
                self.losses.append(loss)
            
            
            epoch_loss+=loss
            epoch_acc+=acc
            
            k=k+1
            if k%self.print_every == 0:
                if k:
                    print (f'Batch:{k} {epoch_loss/(k)}  {epoch_acc/(k)}')  
                    torch.cuda.empty_cache()
        epoch_loss=epoch_loss/len(iterator)
        epoch_acc=epoch_acc/len(iterator)
            
        return epoch_loss,epoch_acc
    
    def plot_lrs(self, n_roll=1):
        import seaborn as sns
        ax=sns.lineplot(x=self.lrs,y=pd.Series(self.losses).rolling(n_roll).mean())
        ax.set_xscale('log')
        ax.set_ylabel('Loss')
        ax.set_xlabel('Learning Rate')

     
    def run_epochs(self,dltrain,dlvalid,n_epochs=1):
        
        if self.cycle_mult > 0:
            reset_cycle=self.cycle_mult
        
        for epoch in range(n_epochs):                

            self.n_epoch+=1
            loss,acc=self.run_epoch(dltrain,True)
            lossv,accv=self.run_epoch(dlvalid,mode_train=False)
            print (f'Epoch:{epoch} Learning rate {self.lr} Weight Decay {self.wd} Train Loss:{loss} Train Accuracy:{acc} Valid Loss:{lossv} Valid Accuracy:{accv}')
        
            if self.cycle_mult:
                if self.n_epoch==reset_cycle:
                    self.lr=self.start_lr
                    #self.wd=self.start_wd
                    reset_cycle=self.n_epoch+reset_cycle
                else:
                    self.lr*=(self.lr_decay**self.n_epoch)  
                    if self.n_epoch>1:
                        self.wd*=self.wd_mult
                
                
            for param_group in self.optimizer.param_groups:
                param_group['lr']=self.lr
                param_group['weight_decay']=self.wd

In [72]:
device="cuda:1"

In [73]:
n_layers

3

In [74]:
model_sentiment=sentiment_classifier (n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e,dropout,\
                 dropout_o,pretrain_mtx=pretrained_lm_weights,n_out=1,padding_idx=1)
model_sentiment=model_sentiment.to(device)

initializing glove with (59972, 400)


In [75]:
optimizer=torch.optim.Adam(model_sentiment.parameters(),lr=5e-3,betas=(0.9,0.999), weight_decay=wd)
#optimizer=torch.optim.SGD(model_sentiment.parameters(),lr=1e-2,momentum=0.9, weight_decay=wd)
metric_fn=accuracy_binomial

In [76]:
len(dltrain),len(dlvalid)

(481, 481)

In [77]:
df_valid['label'].value_counts()

1    12500
0    12500
Name: label, dtype: int64

In [82]:
learner=Learner(model_sentiment,optimizer,accuracy_binomial,device,bptt,100,0.25,cycle_mult=6)

In [83]:
model_sentiment.freeze_embedding()

In [84]:
model_sentiment.encoder.weight.requires_grad, learner.model.encoder.weight.requires_grad

(False, False)

In [85]:
model_sentiment.dropout_e,model_sentiment.dropout,model_sentiment.dropout_o, learner.model.dropout_e,learner.model.dropout,learner.model.dropout_o

(0.2, 0.5, 0.5, 0.2, 0.5, 0.5)

In [86]:
learner.run_epochs(dltrain,dlvalid,5)

Batch:100 0.6561590895056725  0.6403846392035484
Batch:200 0.5366727832704783  0.7254807962477208
Batch:300 0.4839458031952381  0.7623077196876208
Batch:400 0.4557170709222555  0.7820192597061396
Batch:100 0.2724725313484669  0.8825000357627869
Batch:200 0.26886157132685184  0.8884615758061409
Batch:300 0.2781013127664725  0.8832051652669907
Batch:400 0.28134386125952004  0.8809134976565838
Epoch:0 Learning rate 0.005 Weight Decay 1e-07 Train Loss:0.43928361561714735 Train Accuracy:0.7931153341167434 Valid Loss:0.2816500296030124 Valid Accuracy:0.8814249521977192
Batch:100 0.31812149576842785  0.8653846484422684
Batch:200 0.3137647982314229  0.8698077249526978
Batch:300 0.311615818887949  0.8701282382011414
Batch:400 0.3130276659503579  0.8705288803577423
Batch:100 0.24592029325664044  0.9063461929559707
Batch:200 0.24146697673946618  0.9049038848280907
Batch:300 0.24663610418637594  0.8998718329270681
Batch:400 0.24899925781413912  0.8988461922109127
Epoch:1 Learning rate 0.0034999999

KeyboardInterrupt: 

In [85]:
learner.run_epochs(dltrain,dlvalid,2)

Batch:100 0.30271638438105586  0.8767308050394058
Batch:200 0.3132958155870438  0.872692343890667
Batch:300 0.3159764149536689  0.8702564458052318
Batch:400 0.3201662853360176  0.8679327280819416
Batch:100 0.2887049153447151  0.8776923453807831
Batch:200 0.2814566734805703  0.8803846517205238
Batch:300 0.28890246905386446  0.8757051644722621
Batch:400 0.29719005519524216  0.8715865743160248
Epoch:0 Learning rate 0.005 Weight Decay 8.1e-06 Train Loss:0.3237553465948779 Train Accuracy:0.8664761269910419 Valid Loss:0.31267792787286697 Valid Accuracy:0.8644970768206829


KeyboardInterrupt: 

In [91]:
torch.save(model_sentiment.state_dict(),f'{PATH}/inter/master_plannery_nounfreeze_state_dict_0.926')
torch.save(optimizer.state_dict(),f'{PATH}/inter/master_plannery_nounfreeze_state_dict_0.926')
torch.save (model_sentiment,f'{PATH}/inter/master_plannery_nounfreeze_0.926')
torch.save (optimizer,f'{PATH}/inter/master_plannery_nounfreeze_0.926')
torch.save (learner,f'{PATH}/inter/master_plannery_nounfreeze_0.926')

/home/kirana/anaconda3/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type sentiment_classifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [92]:
learner.run_epochs(dltrain,dlvalid,1)

Batch:100 0.1861233239620924  0.9290384995937347
Batch:200 0.2040072415024042  0.9222115752100944
Batch:300 0.20834294127921263  0.9191026026010514
Batch:400 0.20797092800959946  0.9189904239773751
Batch:100 0.19662061750888823  0.9232692706584931
Batch:200 0.19518430452793836  0.9242308068275452
Batch:300 0.19698684968054295  0.9219231156508128
Batch:400 0.19752246107906102  0.9215385001897812
Epoch:0 Learning rate 0.005 Weight Decay 8.1e-06 Train Loss:0.21063770214386146 Train Accuracy:0.917831480255246 Valid Loss:0.19994267463498205 Valid Accuracy:0.9206981033148736


In [102]:
torch.save(model_sentiment.state_dict(),f'{PATH}/inter/master_plannery_nounfreeze_state_dict_0.924')
torch.save(optimizer.state_dict(),f'{PATH}/inter/master_plannery_nounfreeze_state_dict_0.924')
torch.save (model_sentiment,f'{PATH}/inter/master_plannery_nounfreeze_0.924')
torch.save (optimizer,f'{PATH}/inter/master_plannery_nounfreeze_0.924')
torch.save (learner,f'{PATH}/inter/master_plannery_nounfreeze_0.924')

/home/kirana/anaconda3/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type sentiment_classifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [93]:
model_sentiment.unfreeze_embedding()

In [94]:
model_sentiment.encoder.weight.requires_grad, learner.model.encoder.weight.requires_grad

(True, True)

In [95]:
learner.run_epochs(dltrain,dlvalid,2)

Batch:100 0.1648136916011572  0.9338461947441101
Batch:200 0.18203133828938006  0.9277885025739669
Batch:300 0.1935055408626795  0.9226923481623331
Batch:400 0.19548907152377068  0.9235577322542667
Batch:100 0.21496307700872422  0.9142308104038238
Batch:200 0.20938672099262476  0.9173077315092086
Batch:300 0.21482633091509343  0.9149359367291132
Batch:400 0.21724978463724257  0.9122596545517445
Epoch:0 Learning rate 0.0025 Weight Decay 7.290000000000001e-05 Train Loss:0.19839880052419562 Train Accuracy:0.9213617860403477 Valid Loss:0.22030734810629654 Valid Accuracy:0.9104070431725151
Batch:100 0.13987691268324853  0.948653883934021
Batch:200 0.1612088291347027  0.938750039935112


KeyboardInterrupt: 

In [179]:
len(dltrain)

433